In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os


In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Hyperparameters
latent_dim = 100
batch_size = 64
epochs = 50
lr = 0.0002
image_dir = "gan_images"


In [4]:

# Create image directory
os.makedirs(image_dir, exist_ok=True)

In [5]:
# Data loader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Scale images to [-1, 1]
])

In [6]:
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./data', train=True, download=True, transform=transform),
    batch_size=batch_size,
    shuffle=True
)


100%|██████████| 9.91M/9.91M [00:00<00:00, 17.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 477kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.46MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.87MB/s]


In [7]:



# Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 28 * 28),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img.view(z.size(0), 1, 28, 28)


In [8]:

# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        flat = img.view(img.size(0), -1)
        return self.model(flat)


In [9]:

# Initialize models
generator = Generator().to(device)
discriminator = Discriminator().to(device)


In [10]:

# Loss and optimizers
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))


In [11]:

# Training
for epoch in range(epochs):
    for i, (imgs, _) in enumerate(dataloader):
        real_imgs = imgs.to(device)
        valid = torch.ones(imgs.size(0), 1, device=device)
        fake = torch.zeros(imgs.size(0), 1, device=device)

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()
        z = torch.randn(imgs.size(0), latent_dim, device=device)
        gen_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        # Logging
        if i % 200 == 0:
            print(f"[Epoch {epoch}/{epochs}] [Batch {i}/{len(dataloader)}] "
                  f"[D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")

    # Save generated images
    save_image(gen_imgs.data[:25], f"{image_dir}/{epoch:03d}.png", nrow=5, normalize=True)
print("Training complete.")

[Epoch 0/50] [Batch 0/938] [D loss: 0.6849] [G loss: 0.7098]
[Epoch 0/50] [Batch 200/938] [D loss: 0.4949] [G loss: 0.7402]
[Epoch 0/50] [Batch 400/938] [D loss: 0.2340] [G loss: 1.2070]
[Epoch 0/50] [Batch 600/938] [D loss: 0.2481] [G loss: 1.2098]
[Epoch 0/50] [Batch 800/938] [D loss: 0.0888] [G loss: 2.3117]
[Epoch 1/50] [Batch 0/938] [D loss: 0.0760] [G loss: 2.0366]
[Epoch 1/50] [Batch 200/938] [D loss: 0.0745] [G loss: 2.3250]
[Epoch 1/50] [Batch 400/938] [D loss: 0.1769] [G loss: 1.2152]
[Epoch 1/50] [Batch 600/938] [D loss: 0.0843] [G loss: 2.5524]
[Epoch 1/50] [Batch 800/938] [D loss: 0.1210] [G loss: 4.0576]
[Epoch 2/50] [Batch 0/938] [D loss: 0.1034] [G loss: 2.7133]
[Epoch 2/50] [Batch 200/938] [D loss: 0.0646] [G loss: 3.2116]
[Epoch 2/50] [Batch 400/938] [D loss: 0.1206] [G loss: 1.6659]
[Epoch 2/50] [Batch 600/938] [D loss: 0.0539] [G loss: 4.0423]
[Epoch 2/50] [Batch 800/938] [D loss: 0.1896] [G loss: 1.7076]
[Epoch 3/50] [Batch 0/938] [D loss: 0.5081] [G loss: 0.4885]
